In [1]:
import numpy as np
from scipy import stats
import logging
import pandas as pd
import get_redshift

In [2]:
def shift_spectrum(spectrum, base_redshift, max_error_percentage=1):
    """
    Desplaza el espectro según un redshift modificado por un porcentaje aleatorio.
    
    Parameters:
    - spectrum (array): El espectro de la supernova.
    - base_redshift (float): Redshift de referencia de la supernova.
    - max_error_percentage (float): Máximo porcentaje de error en el redshift (default: 0.1).
    
    Returns:
    - array: Espectro desplazado.
    """
    # Genera un número aleatorio entre 0 y 1
    random_factor = np.random.uniform(0, 1)
    # Calcula el redshift modificado con el factor aleatorio
    shift_redshift = base_redshift * (1 + random_factor * max_error_percentage)
    
    # Desplazamos el espectro usando el redshift modificado
    shifted_spectrum = np.interp(
        np.arange(len(spectrum)) * (1 + shift_redshift - base_redshift),
        np.arange(len(spectrum)),
        spectrum,
        left=0,
        right=0
    )
    
    logging.info(f'------Redshift modificando (factor aleatorio): {shift_redshift}')
    return shifted_spectrum

In [3]:
logging.basicConfig(filename='modifications.txt', level=logging.INFO,
                    format='%(asctime)s - %(message)s', filemode='w')

In [4]:
url = "https://github.com/juanserrano90/codelatam/raw/main/Data/data/sn_data.parquet"
df_raw = pd.read_parquet(url)

In [5]:
wavelength = np.array([float(c) for c in df_raw.columns[5:]])

In [6]:
# Configuración de datos
np.random.seed(3312)
column_names = df_raw.columns.values.tolist()
column_names = np.array(column_names[5:]).astype(float)
sn_name = "sn2007uy"
delay_phases = [12.82, 45.82, 54.82, 75.82, 141.82]

# Obtener el redshift desde el archivo CSV usando el módulo
base_redshift = get_redshift.get_redshift(sn_name)

# Obtener y desplazar espectros
sample = []
shifted = []
for delay in delay_phases:
    spectrum = df_raw.loc[(df_raw.index == sn_name) & (df_raw["Spectral Phase"] == delay)].iloc[:, 5:].values[0]
    sample.append(spectrum)
    
    # Aplicar redshift usando el valor obtenido
    shifted_spectrum = shift_spectrum(spectrum, base_redshift)
    shifted.append(shifted_spectrum)

# Crear gráficos
fig, ax = plt.subplots(2, 5, figsize=(20, 10))

# Fila 0: Espectros originales
for i in range(5):
    ax[0, i].plot(column_names, sample[i])

# Fila 1: Espectros con redshift aplicado
for i in range(5):
    ax[1, i].plot(column_names, shifted[i])

# Configuración de gráficos
fig.suptitle(f'Spectra comparison of {sn_name}', fontsize=16)
ax[0, 2].set_title('Original Spectra', fontsize=14, loc='center', pad=20)
ax[1, 2].set_title('Redshifted Spectra', fontsize=14, loc='center', pad=20)

for i in range(5):
    ax[1, i].text(0.5, -0.2, f'Phase delay: {delay_phases[i]}', ha='center', va='center', transform=ax[1, i].transAxes, fontsize=12)

plt.tight_layout(rect=[0, 0, 1, 0.95])
plt.show()

FileNotFoundError: Error: The file /Data/Redshift/sn_redshift.csv was not found.